<a href="https://colab.research.google.com/github/H4D32/AndrewNg-ML-Python-Notebooks/blob/main/Ex4/AndrewEx4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import scipy.optimize as op

In [34]:
mat=loadmat("ex4data1.mat")

X=mat["X"]
y=mat["y"]

X.shape

(5000, 400)

In [35]:
mat2=loadmat("ex4weights.mat")
Theta1=mat2["Theta1"] 
Theta2=mat2["Theta2"] 

print(Theta1.shape)
print(Theta2.shape)

(25, 401)
(10, 26)


In [36]:
def sigmoid(z):
  """
  Compute the sigmoid of each value of z
  """

  return 1 / (1 + np.exp(-z))

In [37]:
def sigmoidGradient(z):
    """
    computes the gradient of the sigmoid function
    """
    
    sigmoid = 1/(1 + np.exp(-z))
    
    return sigmoid *(1-sigmoid) 

In [38]:
def nnCostFunction(nn_params,input_layer_size, hidden_layer_size, num_labels,X, y,Lambda):

    # Reshape nn_params back into the parameters Theta1 and Theta2, the weight matrices
    # for our 2 layer neural network
    Theta1 = nn_params[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
    Theta2 = nn_params[((input_layer_size +1)* hidden_layer_size ):].reshape(num_labels,hidden_layer_size+1)

    m,n = X.shape
    J = 0
    Theta1_grad = np.zeros((Theta1.shape))
    Theta2_grad = np.zeros((Theta2.shape))

    X = np.hstack((np.ones((m,1)),X))
    a1 = X
    
    z2 = a1 @ Theta1.T
    a2 = sigmoid(z2)
    a2 = np.hstack((np.ones((m,1)), a2))

    z3 = a2 @ Theta2.T
    a3 = sigmoid(z3)

    h_x = a3
    y_vec = np.zeros((m,num_labels))

    for i in range(1,num_labels+1):
        y_vec[:,i-1][:,np.newaxis] = np.where(y==i,1,0)
    y_vec_F = y_vec.flatten()
    h_x_F = h_x.flatten()

    #J = (-y_vec(:)' * log(h_x(:)) - (1-y_vec(:))' * log(1-h_x(:)))/m
    J = (-y_vec_F.T @ np.log(h_x_F) - (1-y_vec_F).T @ np.log(1-h_x_F))/m

    newTheta1 = Theta1[:,1:].flatten()
    newTheta2 = Theta2[:,1:].flatten()

    reg_term = Lambda*((newTheta1.T @ newTheta1) + newTheta2.T @ newTheta2)
    reg_J = J + (1/(2*m))*reg_term

    for i in range(m):
        xi= X[i,:] # 1 X 401
        a2i = a2[i,:] # 1 X 26
        a3i =a3[i,:] # 1 X 10
        d3 = a3i - y_vec[i,:]
        d2 = Theta2.T @ d3.T * sigmoidGradient(np.hstack((1,xi @ Theta1.T)))
        Theta1_grad= Theta1_grad + d2[1:][:,np.newaxis] @ xi[:,np.newaxis].T
        Theta2_grad = Theta2_grad + d3.T[:,np.newaxis] @ a2i[:,np.newaxis].T
        
    Theta1_grad = 1/m * Theta1_grad
    Theta2_grad = 1/m*Theta2_grad
    
    Theta1_grad_reg = Theta1_grad + (Lambda/m) * np.hstack((np.zeros((Theta1.shape[0],1)),Theta1[:,1:]))
    Theta2_grad_reg = Theta2_grad + (Lambda/m) * np.hstack((np.zeros((Theta2.shape[0],1)),Theta2[:,1:]))

    return J, Theta1_grad, Theta2_grad, reg_J, Theta1_grad_reg, Theta2_grad_reg

In [39]:
input_layer_size  = 400
hidden_layer_size = 25
num_labels = 10
nn_params = np.append(Theta1.flatten(),Theta2.flatten())
J, Theta1_grad, Theta2_grad, reg_J, Theta1_grad_reg, Theta2_grad_reg = nnCostFunction(
    nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, 1)
print("Cost at parameters (non-regularized):",J,"\nCost at parameters (Regularized):",reg_J)

Cost at parameters (non-regularized): 0.2876291651613189 
Cost at parameters (Regularized): 0.38376985909092365


In [40]:
def randInitializeWeights(L_in, L_out):
    """
    randomly initializes the weights of a layer with L_in incoming connections and L_out outgoing connections.
    Note that W should be set to a matrix of size(L_out, 1 + L_in) as
    the first column of W handles the "bias" terms
    """
    
    epi = (6**1/2) / (L_in + L_out)**1/2
    
    W = np.random.rand(L_out,L_in +1) *(2*epi) -epi
    
    return W

In [41]:
initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)
initial_nn_params = np.append(initial_Theta1.flatten(),initial_Theta2.flatten())

In [42]:
def nnpredict(Theta1, Theta2, X):
    """
    Predict the label of an input given a trained neural network
    """
    m= X.shape[0]
    a1 = np.hstack((np.ones((m,1)),X))
    z2 = a1 @ Theta1.T
    a2 = sigmoid(z2)
    a2 = np.hstack((np.ones((m,1)), a2))
    z3 = a2 @ Theta2.T
    a3 = sigmoid(z3)

    return np.argmax(a3,axis=1)+1

In [43]:
def gradientDescentnn(X,y,initial_nn_params,alpha,num_iters,Lambda,input_layer_size, hidden_layer_size, num_labels):
    """
    Take in numpy array X, y and theta and update theta by taking num_iters gradient steps
    with learning rate of alpha
    
    return theta and the list of the cost of theta during each iteration
    """
    Theta1 = initial_nn_params[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
    Theta2 = initial_nn_params[((input_layer_size +1)* hidden_layer_size ):].reshape(num_labels,hidden_layer_size+1)
    
    m=len(y)
    J_history =[]
    
    for i in range(num_iters):
        nn_params = np.append(Theta1.flatten(),Theta2.flatten())
        cost, grad1, grad2 = nnCostFunction(nn_params,input_layer_size, hidden_layer_size, num_labels,X, y,Lambda)[3:]
        Theta1 = Theta1 - (alpha * grad1)
        Theta2 = Theta2 - (alpha * grad2)
        J_history.append(cost)
    
    nn_paramsFinal = np.append(Theta1.flatten(),Theta2.flatten())
    return nn_paramsFinal , J_history

In [44]:
nnTheta, nnJ_history = gradientDescentnn(X,y,initial_nn_params,0.8,800,1,input_layer_size, hidden_layer_size, num_labels)
Theta1 = nnTheta[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
Theta2 = nnTheta[((input_layer_size +1)* hidden_layer_size ):].reshape(num_labels,hidden_layer_size+1)

In [45]:
pred3 = nnpredict(Theta1, Theta2, X)
print("Training Set Accuracy:",sum(pred3[:,np.newaxis]==y)[0]/5000*100,"%")

Training Set Accuracy: 94.14 %
